# Cost overview

- parse usage output and calculate costs for an example

- [ ] Check prices for CR2

In [ ]:
import pprint
import pathlib

import yaml

import pandas as pd

TESTFILE = pathlib.Path('logs/') / 'usage_2022_02_10.txt'

fpath = TESTFILE
with open(fpath) as f:
    report = next(f)
    report += next(f)
    headers = [filter(None, next(f).strip().split('  ')[1:])]
    headers.append(filter(None, (next(f).strip().split('  '))))
    headers = [f'{header_1.strip()} {header_2.strip()}' for header_1, header_2 in  zip(*headers)]
    _ = next(f)
    data = {}
    for row in f:
        if row.startswith('---'):
            break
        line = row.split()
        data[line[0]] = {k:float(v) for k, v in zip(headers, line[1:])}
    total = next(f).split()
    storage = f.read().strip()

print(report, headers, total, storage, sep='\n')

pprint.pprint(data)

In [ ]:
storage_gb = int(storage.split(':')[1].split()[0])
storage_gb

In [ ]:
# {key.strip(): float(value) for key, value in zip(headers, total[1:])}
_entry = {}
for key, value in zip(headers, total[1:]):
    try:
        _entry[key.strip()] = int(value)
    except ValueError:
        _entry[key.strip()] = float(value)
data[total[0].strip(':')] = _entry


In [ ]:
df = pd.DataFrame(data).T
df

In [ ]:
with open('cr2_prices.yaml') as f:
    prices = yaml.safe_load(f)
prices

In [ ]:
print(f"Storage costs: {prices['storage_TB']*storage_gb/1024:.2f} DKK")

In [ ]:
costs = df[headers[0]] * prices['thin_node'] / 40
costs = costs.to_frame('thin nodes')
costs['fat nodes'] = df[headers[2]] * prices['fat_node'] / 80
costs['gpu nodes'] = df[headers[4]] * prices['gpu_node']
costs['total'] = costs.sum(axis=1)
costs